# **Multiclass Fish Image Classification**

# **Libraries Used**

In [26]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models,layers

# **Data Loading**

Train Data:

In [27]:
# Train Data path
train_data_path = "data/train"

# All Classes
classes = os.listdir(train_data_path)

# Ignore "Animal fish" and "Animal fish bass"
classes_filtered = [cls for cls in classes if cls.startswith("fish sea_food")]

# Keeping only Species name 
classes_new = [cls.replace("fish sea_food ", "") for cls in classes_filtered]
classes_final = [cls.replace("_", " ").title() for cls in classes_new]

# Rescaling
rescaled = ImageDataGenerator(rescale=1./255)

# Load Train Data
train_generator = rescaled.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode="categorical"
)

Found 5099 images belonging to 9 classes.


Validation Data:

In [28]:
# Validation Data path
val_data_path = "data/val"

# Load Validation Data
val_generator = rescaled.flow_from_directory(
    val_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode='categorical'
)

Found 895 images belonging to 9 classes.


Test Data:

In [29]:
# Test Data path
test_data_path = "data/test"

# Load Test Data
test_generator = rescaled.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode='categorical',
    shuffle=False
)

Found 2654 images belonging to 9 classes.


Label Mapping:

In [30]:
# Label Mapping 
index_to_label = {i: name for i, name in enumerate(classes_final)}

# Class Mapped
print("Class index mapping:", index_to_label, sep='\n')

Class index mapping:
{0: 'Black Sea Sprat', 1: 'Gilt Head Bream', 2: 'Hourse Mackerel', 3: 'Red Mullet', 4: 'Red Sea Bream', 5: 'Sea Bass', 6: 'Shrimp', 7: 'Striped Red Mullet', 8: 'Trout'}
